In [5]:
import pandas as pd
import numpy as np

df_lsoa = pd.read_parquet("../data/processed/census_lsoa.parquet")

lookup = pd.read_csv(
    "../data/lookups/look up LSOA 2021 to ward 2024 merged.csv",
    usecols=["LSOA21CD", "WD24CD", "WD24NM"]
)

# Mege on LSOA code
df_lsoa = df_lsoa.merge(
    lookup,
    left_on="LSOA code",
    right_on="LSOA21CD",
    how="inner"
)
df_ward_census = (
    df_lsoa
    .groupby(["WD24CD", "WD24NM"], as_index=False)
    .agg({"Total: All dwellings": "sum"})
)
df_ward_census.rename(
    columns={
        "WD24CD": "Ward code",
        "WD24NM": "Ward name",
        "Total: All dwellings": "Dwellings"
    },
    inplace=True
)

sarima_fc = pd.read_csv(
    "../data/processed/sarima_final_forecast_per_ward.csv",
    parse_dates=["Month"]
)
first_fc_month = sarima_fc["Month"].min()
fc1 = sarima_fc[sarima_fc["Month"] == first_fc_month].copy()

df = (
    fc1[["Ward code", "Ward name", "mean"]]
    .merge(
        df_ward_census[["Ward code", "Dwellings"]],
        on="Ward code",
        how="left"
    )
    .rename(columns={"mean": "Forecasted_Burglaries"})
)

df["Risk_Rate"] = df["Forecasted_Burglaries"] / df["Dwellings"].replace(0, np.nan)
df["Risk_Rate"] = df["Risk_Rate"].clip(lower=0)

r_max = df["Risk_Rate"].max()

df["Allocated_Hours"] = 200 * (df["Risk_Rate"] / r_max)
df["Allocated_Hours"] = df["Allocated_Hours"].clip(lower=2)

df["Allocated_Hours"] = df["Allocated_Hours"].round(2)
df = df.sort_values("Allocated_Hours", ascending=False).reset_index(drop=True)

print(df[[
    "Ward code",
    "Ward name",
    "Dwellings",
    "Risk_Rate",
    "Allocated_Hours"
]])

df.to_csv("../data/processed/ward_hour_allocation.csv", index=False)


     Ward code                 Ward name  Dwellings  Risk_Rate  \
0    E05013662   Holborn & Covent Garden       5985   0.004386   
1    E05013748     Shepherd's Bush Green       3080   0.004309   
2    E05009377  Hoxton East & Shoreditch       6410   0.003821   
3    E09000001            City of London       7330   0.003821   
4    E05013808                  West End      10270   0.003786   
..         ...                       ...        ...        ...   
675  E05014078    Eltham Park & Progress       4640   0.000041   
676  E05013751           West Kensington       5210   0.000037   
677  E05013673                     Bowes       4115   0.000000   
678  E05013513                Welsh Harp       6360   0.000000   
679  E05013578             Ruislip Manor       3920   0.000000   

     Allocated_Hours  
0             200.00  
1             196.52  
2             174.25  
3             174.24  
4             172.66  
..               ...  
675             2.00  
676             2.00  
